<a href="https://colab.research.google.com/github/RyanTokManMokMTM/NLP_Training4/blob/main/Words2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip wiki_zh_2019
#unzip the zipfile

In [ ]:
from os import listdir
from os.path import isfile, isdir, join
import json
ALL_DATAS = []
filesList = {
    0:"AA",
    1:"AB",
    2:"AC",
    4:"AD",
    5:"AE",
    6:"AF",
    7:"AG",
    8:"AH",    
    9:"AI",
    10:"AJ",
    11:"AK",
    12:"AL",
    13:"AM"
}

for i in filesList:
  folderPath = "./wiki_zh/%s/" % filesList[i]
  files = listdir(folderPath)
  for i in files:
    curFilePath = folderPath + "%s"%i 
    print(curFilePath)
    with open(curFilePath) as f:
      for j in f.readlines():
        str2Json = json.loads(j)
        text = "".join(str2Json["text"].strip().split())
        ALL_DATAS.append(text)
      f.close()


In [ ]:
!pip install opencc-python-reimplemented

In [ ]:
#convert text 
from opencc import OpenCC
#convert to zh-hk
ccConvertor = OpenCC("s2hk")
transChineseList = []
for i in ALL_DATAS:
  transChineseList.append(ccConvertor.convert(i))


In [ ]:
#store convertd data to csv
import pandas as pd
datas = pd.DataFrame(transChineseList)
datas.to_csv("alldatas.csv",encoding="utf-8")

# cut words

In [ ]:
!pip install jieba

In [ ]:
#load stopwords list
import codecs 
fp = codecs.open("./stopwords.dat","r",encoding="utf-8")
contents = fp.read()
fp.close()

In [ ]:
import jieba
import jieba.analyse
cutWordsListDatas = []
x = 0;
for i in transChineseList:
  cutWordsListDatas.append(" ".join([word for word in jieba.cut(i,cut_all=False) if word not in contents]))
  if(x%5000 == 0):
    print(x)
  x+=1

In [66]:
cutWordsTxt = "cutWordsList2.txt"
with open(cutWordsTxt,"w",encoding = "utf-8") as fw:
  for data in cutWordsListDatas:
    fw.write(data)
    fw.write("\n")


In [93]:
cutWordsListDatas[0]

'固定翼 飛機 固定翼 飛機 簡稱 定翼機 簡稱 飛機 英文 指由動力 裝置產 生前 推力 拉力 機身 固定 機翼產生 升力 大氣層 飛行 空氣 航空器 固定翼 航空器 一種 最常見 一種 一種 固定翼 航空器 滑翔 飛機 發動 機類 型 分為 噴氣 飛機 螺旋 槳 飛機 固定翼 機是 人類 20 世紀 科學 技術 成就 有人 將它 電視 電腦 並列 20 世紀 人類 影響 三大 發明 世界 最早 固定翼 發明 各國 尚存 爭議 觀點 美國 人萊特 兄弟 發明 1903 12 17 飛行 第一次 空氣 航空器 受控 持續 動力 飛行 國際 航空 聯合會 FAI 所認 交通工具 相比 固定翼 機有 優點 固定翼 機作 交通工具 侷 限性 固定翼 機不斷 提升 燃油 效率 飛行 消耗 公斤 燃油 排放 公斤 二氧化碳 氣體 大氣層 排放 全球 暖化 影響 地面 排放 飛機 機體 結構 包括 機翼 機身 尾翼 起落架 飛機 發動機 機身 則發動 機短艙 也屬 機體 結構 一部分 機翼 飛 機產生 升力 部件 機翼後緣 可操 活動面 外側 副翼 控制 飛機 滾 轉運動 內側 的則 襟翼 增加 起飛 着 陸階段 升力 著陸時 阻力 提供 更大 下降 坡率 機翼 內部 安裝 油箱 機翼 下面 則可供 掛載 副油箱 武器 附加 設備 飛機 發動機 起落架 安裝 機翼 下方 大型 飛機 翼面 設有擾 流板 飛行 轉向 降低 升力 輔助 轉向 亦可 飛行 降落 接地 時立 減速 機翼 形狀 數目 航空 技術 發達 早期 提供 更大 升力 固定翼 機以 雙翼機 多翼 機為 飛機 單翼機 機身 功用 裝載 人員 貨物 設備 燃料 武器 飛機 結構 部件 安裝 基礎 尾翼 機翼及 發動機 連接成 整體 飛翼機 機身 隱藏 機翼 內部 尾翼 平衡 穩定 操縱 飛機 飛行 姿態 部件 包括 垂直 尾翼 垂尾 水平尾翼 平尾 垂直 尾翼 固定 垂直 安定 面和安裝 其後部 方向舵 水平尾翼 固定 水平 安定 面和安裝 其後部 升降舵 型號 飛機 升降舵 全動式 水平尾翼 方向舵 小幅度 控制 修正 飛機 航向 運動 升降舵 控制 飛機 俯仰 運動 起落架 支撐 飛機 停放 滑行 起飛 着 陸 滑跑 部件 支柱 緩衝器 剎車 裝置 機輪 收放 機構 陸上 飛機 起落 裝置 減震 支柱 機輪 還有專

# Word2vec for our text

In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [55]:
#store txt to array\
cutWordsListDatas = []
with open("/content/drive/MyDrive/cutWordsList.txt") as f:
  for i in f.readlines():
    cutWordsListDatas.append(i.rstrip("\n"))

In [ ]:
!pip install gensim

In [48]:
!pip install word2vec

In [ ]:
import word2vec
word2vec.word2vec("cutWordsList2.txt","demo.bin",size=300,verbose=True,binary=True)

In [69]:
model =  word2vec.load("demo.bin")

In [136]:
if __name__ == '__main__':
  word = "鋼鐵人"
  datas = model.similar(word,n=20)
  word = list(datas[0])
  score = list(datas[1])
  for i in range(len(word)):
      print("%s : %s" % (model.vocab[word[i]],score[i]))


蜘蛛人 : 0.7715826053998335
水行俠 : 0.737633098643578
仇者 : 0.7172986328173914
鋼鐵俠 : 0.7097133364879444
夜魔俠 : 0.6916923892783963
死侍 : 0.6905602600709029
漫威 : 0.6771899330465423
浩克 : 0.6771353180448477
閃電俠 : 0.6745832078566145
綠燈俠 : 0.6715115775461686
特攻隊 : 0.6641374024557118
戰甲 : 0.6584346089582496
鐵拳俠 : 0.654428165528822
綠箭俠 : 0.6527442821540076
女超人 : 0.648465130172418
魔俠 : 0.6450306426433308
超人 : 0.6397801118649373
魔龍 : 0.6333124260608268
前傳 : 0.6330088338004289
美國隊 : 0.629012049371652
